In [3]:
from datasets import load_dataset
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas
import openai
from openai import APIError
import os
import json
import re
import numpy as np
from sklearn.cluster import KMeans
from pprint import pprint
from pathlib import Path
import tiktoken




# Load the .env file
load_dotenv()

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY


In [4]:
dir = Path("~/Datasets/SRBendding").expanduser()
dir.mkdir(parents=True, exist_ok=True)
dir

PosixPath('/home/selena/Datasets/SRBendding')

In [5]:
dataset = load_dataset("jerteh/SrpWiki", cache_dir=dir)
dataset2 = load_dataset("jerteh/SrpKorNews",  cache_dir=dir)


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3818535
    })
})

In [7]:
dataset['train']

Dataset({
    features: ['text'],
    num_rows: 3818535
})

In [8]:
wikiText = dataset['train']['text']
len(wikiText)

3818535

In [9]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [11]:
jump = 1
new_contexts = []
mini = float('inf')
maxi = 0
total_tokens = 0
num_contexts = 0
num_bigger_than_128 = 0

for i in range(0, len(wikiText) - jump, jump):
    context = ' '.join(wikiText[i:i+jump])
    token_num = num_tokens_from_string(context, "cl100k_base")
    
    # Update min, max, and total tokens
    mini = min(token_num, mini)
    maxi = max(token_num, maxi)
    
    total_tokens += token_num
    num_contexts += 1

    if token_num > 128:
        num_bigger_than_128 += 1

# Calculate the average token count
average_tokens = total_tokens / num_contexts if num_contexts > 0 else 0

# Output the results
print(f"Minimum token count: {mini}")
print(f"Maximum token count: {maxi}")
print(f"Average token count: {average_tokens}")
print(f"counts bigger than 128: {num_bigger_than_128}")

Minimum token count: 2
Maximum token count: 51022
Average token count: 46.49088681677314
counts bigger than 128: 54316


In [7]:
for i in range(10):
    print(wikiText[i])

UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.
Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.
U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet.
Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita).
To je bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard.
1968. godine je skup karaktera proširen na 256 (8 bita), a "gornjih" 128 karaktera je bilo korišćeno za dodatne karaktere.
Iz neke navike je i ovaj prošireni "-{ASCII}-" nazivan "-{ASCII}-", tako da tu često dolazi do zabune.
Da bi postojala podrška za više jezika, smišljane su tzv. kodne strane ("-{Code Page}-") koje definišu ponašanje tog dodatnog skupa slov

In [37]:
context = "UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet."
PROMPT = """
### Goal ###
Tre primary objective is to produce multiple queries in the Serbian language and a list of keywords in the Serbian language from the provided context. The context repesents an answer to the query and the keywords best 
describe the context. The goal is to have query-context pairs that corelate with each other and a list of keywords that would spead-up the search in the future.

### Process Overview ###
1. Carefully read and analyze the given context text.
2. Identify all relevant keywords and what the context text is about.
3. Find the queries that best represents the given context text.

### Formatting Rules ###
- Keyword value MUST be a LIST of strings with max lenght of 5 or [null] if no relevant information is provided.
- Use double quotes for strings and escape internal quotes with a backslash (\).
- Keep the queries concise and general about the context text.
- Ensure the output is a valid JSON file, parsable by Python's json.loads().
- Strictly use only the information provided in the context text. Do not add, infer, or imagine any additional details beyond what is explicitly stated.
- Remember to answer in Serbian.

### Query description###
- All queries must be WH-questions.
- All queries must begin with a capital letter and end with a questoin mark (?).

### Output Format ###
{{
 "keywords": ["The keyword that best represent the given context with max lenght of 5"],
 "short_query": "A short query that best suits the given context. It should be only a few words long and general.",
 "medium_query": "A minium lenght query that best suits the given context. It should be a lenght of min 10 words and max 18.",
 "long_query": "A long query that best suits the given context. It should be longer than 19 words and very specific to the context."
}}

### Context ###
{context}
"""

In [38]:
PROMPT

'\n### Goal ###\nTre primary objective is to produce multiple queries in the Serbian language and a list of keywords in the Serbian language from the provided context. The context repesents an answer to the query and the keywords best \ndescribe the context. The goal is to have query-context pairs that corelate with each other and a list of keywords that would spead-up the search in the future.\n\n### Process Overview ###\n1. Carefully read and analyze the given context text.\n2. Identify all relevant keywords and what the context text is about.\n3. Find the queries that best represents the given context text.\n\n### Formatting Rules ###\n- Keyword value MUST be a LIST of strings with max lenght of 5 or [null] if no relevant information is provided.\n- Use double quotes for strings and escape internal quotes with a backslash (\\).\n- Keep the queries concise and general about the context text.\n- Ensure the output is a valid JSON file, parsable by Python\'s json.loads().\n- Strictly us

In [41]:

def test_api(prompt, context, model: str = "gpt-3.5-turbo-0125"): # TODO probaj sa miniii
    client = openai


    # Assuming OpenAI API key is set elsewhere
    # Initiate the streaming response
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": context}
        ],
        response_format={'type': 'json_object'},
        temperature=0,
        top_p=1
    )

    return response


In [42]:
res = test_api(PROMPT, context)
returned_obj = res.choices[0].message.content
json.loads(returned_obj)

{'keywords': ['UTF-8',
  'kodiranje',
  'latinični tekst',
  'Microsoft Word',
  'Unicode fontovi'],
 'short_query': 'Koja je najzgodnija varijanta za kodiranje latiničnog teksta?',
 'medium_query': 'Kako se koristi UTF-8 varijanta u Microsoft Word-u i Netscape Composer-u?',
 'long_query': 'Koje su preporučene standardne Unicode fontovi za laku prenosivost teksta sa računara na računar ili za objavljivanje na Internetu?'}

In [56]:
# from openai import OpenAI
# client = OpenAI()

# def normalize_l2(x):
#     x = np.array(x)
#     if x.ndim == 1:
#         norm = np.linalg.norm(x)
#         if norm == 0:
#             return x
#         return x / norm
#     else:
#         norm = np.linalg.norm(x, 2, axis=1, keepdims=True)
#         return np.where(norm == 0, x, x / norm)
    


# def get_embedding(text, model="text-embedding-3-small", dimension=None):
#     text = text.replace("\n", " ")

#     embedded_text = client.embeddings.create(input = [text], model=model).data[0].embedding
#     if not dimension:
#        return embedded_text
#     cut_dim = embedded_text[:dimension]
#     return normalize_l2(cut_dim)


# embedding = get_embedding("recenica akldakjdn jakjaaj jjjjjj", dimension=768)
# len(embedding)
# # 1536
# # 768 je outoutovih srpskih
# # zakljucak, treba redukcija dimenzionalnosti

768

In [24]:

jump = 4
new_contexts = []
for i in range(0, 10 * 3, jump):
    context = ' '.join(wikiText[i:i+jump])
    print(context)
    new_contexts.append(context)
    # print(len(context.split(". ")))
    res = test_api(PROMPT, context)
    returned_obj = res.choices[0].message.content
    pprint(json.loads(returned_obj))



UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate. U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet. Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita).
{'keywords': ['UTF-8', 'kodiranje', 'latinični tekst', 'Unicode', 'fontovi'],
 'long_query': 'Kako su prvi računari podržavali samo engleski alfabet i kako '
               'je to uticalo na razvoj kodiranja teksta u današnje vreme?',
 'medium_query': 'Kako se koristi UTF-8 varijanta za kodiranje teksta u '
                 'Microsoft Word-u, Netscape Composer-u i Kate?',
 'short_query': 'Koja je najzgodnija varija

In [38]:
dataset2

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 337
    })
})

In [40]:
dataset2['train']

Dataset({
    features: ['text'],
    num_rows: 337
})

In [41]:
dataset2['train']['text'][0]

'<s>Mečka na usijanom limenom krovu</s><s>Piše: Dušan Vidaković Kada je austrijski konzul u Nici Emil Jelinek 1901. naručio od nemačkog Dajmlera, odjednom, čak 36 automobila, čija je ukupna cena premašila fantastičnih 550 hiljada zlatnih maraka, imao je samo jedan uslov: da sva ta kola nose ime njegove ćerke.</s><s>Ljudi iz kanstatske firme, naravno, nisu imali ništa protiv, a istorija automobilizma je imala sreću da je Jelinekovoj mezimici kum dao (pogotovo za Špance) vrlo graciozno ime - Mercedes, što na jeziku konkvistadora znaci ljupkost, umilnost, prijatnost.</s><s>Jelinekova porudžbina značila prekretnicu u poslovanju Dajmlera, a naziv Mercedes je preživeo i fuziju sa fabrikom Karla Benca, 1926. godine, održavši se do danas kao sinonim za kvalitetne, luksuzne i, nadasve, bezbedne automobile svojstvenih vozačima koji ne kriju brige u novčaniku.</s><s>Međutim, upravo legenda o blistavosti trokrake zvezde doživljava ovih dana ozbiljnu proveru zahvaljujući mercedesu A klase.</s><s>A 

In [25]:
count = 0
for paragraf in dataset2['train']['text']:
    paragraph_sentences = re.findall(r'<s>(.*?)</s>', paragraf)
    count += len(paragraph_sentences)
    # print(len(sentences))
print(count)


21982152


In [87]:
# for sentence in sentences:
#     print(sentence)

### Making parallel calls

In [51]:
def save_jobs(sentences, filename, prompt_template, model: str = "gpt-3.5-turbo-0125"):    
    jobs = [
                {
                "model": model,
                "response_format": {'type': 'json_object'},
                "temperature": 0,
                "metadata": {"context": sentence},
                "messages": [
                    {
                        "role": "system",
                        "content": prompt_template.format(
                            context=sentence
                        ),
                    }
                ],
            }
                for sentence in sentences
            ]
    with open(filename, "w", encoding='UTF-8') as f:
        for job in jobs:
            json_string = json.dumps(job, ensure_ascii=False)
            f.write(json_string + "\n")


In [52]:
save_jobs(new_contexts, "datasets/training_comands.ljson", PROMPT)

python api_request_parallel_processor.py   --requests_filepath training-dataset/datasets/training_comands.ljson   --save_filepath training-dataset/datasets/training_openai_data.jsonl   --request_url https://api.openai.com/v1/chat/completions   --max_requests_per_minute 1500   --max_tokens_per_minute 6250000   --token_encoding_name cl100k_base   --max_attempts 5   --logging_level 20 



menjaj max token per min na 1_250_000
menjaj max request per min na 4_000


In [84]:


def make_dataset(file_path: Path):
        returned_dict = {
             "context": [],
             "short_query": [],
             "medium_query": [],
             "long_query": [],
             "keywords": []
        }
        # Open and iterate through the .jsonl file
        with open(file_path, 'r') as file:
            for line in file:
                data = json.loads(line)
                context = data[-1]['context']
                returned_data = data[1]['choices'][0]['message']['content']
                returned_data = json.loads(returned_data)
                returned_dict['context'].append(context)
                returned_dict['short_query'].append(returned_data['short_query'])
                returned_dict['medium_query'].append(returned_data['medium_query'])
                returned_dict['long_query'].append(returned_data['long_query'])
                returned_dict['keywords'].append(returned_data['keywords'])
        return returned_dict

In [85]:
data_for_df = make_dataset(Path('datasets/training_openai_data.jsonl'))
dataset = pd.DataFrame(data_for_df)

table = pa.Table.from_pandas(dataset)

pq.write_table(table, 'datasets/train.parquet')

In [86]:
dataset.head()

,context,short_query,medium_query,long_query,keywords
0,Drugo rešenje je korišćenje tzv. transformacio...,Šta je Unicode transformaciona šema?,Kako se koristi Unicode transformaciona šema z...,Kako funkcioniše Unicode transformaciona šema ...,"[Unicode, transformacione šeme, UTF-8, latinica]"
1,"To uglavnom nije problem realizovati, ali ako ...",Šta je Junikod?,Kako se rešava problem mešanja dva pisma u tur...,Kako je nastao Junikod i zašto se koristi kao ...,"[Unicode, pisma, turistički vodič, jezici]"
2,"O UTF-8 će biti više reči u poglavlju ""Ukratko...",Šta omogućavaju MIME standardi?,Kako se vrše kodiranje višebitnih reči u Inter...,Koje su razlike između 7-bitnih mail poruka i ...,"[UTF-8, Mail Transfer Agent, MIME standardi, U..."
3,"To je bio tzv. ""-{ASCII}-"" ili ""-{US-ASCII}-"" ...",Šta je bio tzv. ASCII standard?,Kako je proširen ASCII standard 1968. godine i...,Kako su kodne strane smišljene radi podrške za...,"[ASCII, US-ASCII, standard, kodne strane, kara..."
4,Osnovna kodna strana na personalnim računarima...,Koje kodne strane podržavaju različite jezike ...,Kako se različite kodne strane koriste za podr...,"Kako kodne strane kao što su Latin1, Latin2 i ...","[kodna strana, personalni računari, grafika, k..."


1. ako kazemo da nam je kontext 10 recenica onda za sliding window uzimamo 20  

    - prednost sto onda sigurno eliminisemo  
    - mana sto kad su dobri onda gubimo pola podataka-sto ustvari nije ni mana jer imamo svakako 2m recenica  

In [1]:
dataset.head()

NameError: name 'dataset' is not defined

In [8]:
dataset = Path("~/Datasets/SRBendding").expanduser()
dataset.mkdir(parents=True, exist_ok=True)

dataset

PosixPath('/home/selena/Datasets/SRBendding')

In [ ]:
data = load_dataset("ms", 'v1.1.', cache_dir=dataset)

64
512
gausova raspodela, uzme se random broj odatkle i pravi se gausova kriva

smislimo prag i za manje i za vise


na vecem delu dataset-a chuncking, onda na random delu odbirkovano biramo cankove, random primeri idu na semanic, od tih cankova idemo na generae query

damo chatqpt u promptu da nam da score za query i context recimo od 1-5
embedding, normalizujemo uradimo cosine i ako je dobro idemo dalje